# Lecture 5: Neural Network

# Sonar Case Study : Stochastic Gradient Decent and _k_-fold Cross Validation Approach
Source: _Machine Learning From Scratch_ (Machine Learning Mastery)

### Imports

In [ ]:
# python imports

from random import seed
from random import randrange
from csv import reader

### Function to load the CSV file and create a list of dataset

In [ ]:
# load a CSV file

def load_csv(filename):

	dataset = list()
 
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	 
	return dataset


### str_column_to_float() - converts string to numeric values 

In [ ]:
# convert string column to float

def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())

### str_column_to_int()  -  output column is converted from string to integer values of 0 to 1

In [ ]:
# convert string column to integer

def str_column_to_int(dataset, column):

	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
 
	for i, value in enumerate(unique):
		lookup[value] = i
		
	for row in dataset:
		row[column] = lookup[row[column]]

	return lookup
	

### k-fold Cross Validation Set - estimate the performance of the model

In [ ]:
# split a dataset into k-folds

def cross_validation_split(dataset, n_folds):

	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
 
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	
	return dataset_split

### Evaluate Classification Accuracy of Each Model Using Cross-Validation Split

In [ ]:
# evaluate an algorithm using a cross validation split

def evaluate_algorithm(dataset, algorithm, n_folds, *args):

	folds = cross_validation_split(dataset, n_folds)
	scores = list()
 
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
	
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None

		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	
	return scores

### Estimate the Performance of _k_-models: accuracy

In [ ]:
# calculate accuracy percentage

def accuracy_metric(actual, predicted):

	correct = 0

	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1

	return correct / float(len(actual)) * 100.0

### predict() and train_weights() function to train the model

In [ ]:
# make a prediction with weights

def predict(row, weights):

	activation = weights[0]

	for i in range(len(row)-1):
		activation += weights[i + 1] * row[i]

	return 1.0 if activation >= 0.0 else 0.0

# estimate weights using stochastic gradient descent

def train_weights(train, l_rate, n_epoch):

	weights = [0.0 for i in range(len(train[0]))]

	for epoch in range(n_epoch):
		for row in train: # mini-batch is one row!
			prediction = predict(row, weights)
			error = row[-1] - prediction
			weights[0] = weights[0] + l_rate * error

			for i in range(len(row)-1):
				weights[i + 1] = weights[i + 1] + l_rate * error * row[i]

	return weights


### perpceptron() to tie precit() and train_weights() together

In [ ]:
# train with SGD

def perceptron(train, test, l_rate, n_epoch):

	predictions = list()
	weights = train_weights(train, l_rate, n_epoch)
 
	for row in test:
		prediction = predict(row, weights)
		predictions.append(prediction)
	
	return(predictions)

# Training Network on Sonar Dataset

### Imports to load the csv file 

In [ ]:
# imports to upload the datafile
import io
from google.colab import files  

### Uploade the csv file to local directory

In [ ]:
# to upload local files
uploaded = files.upload()

Saving sonar.all-data.csv to sonar.all-data (1).csv


### Save the daset to working directory 

In [ ]:
# load and prepare data
seed(1)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)

### Prepare data

In [ ]:
# convert string class to float
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0])-1)

{'M': 0, 'R': 1}

### Run perceptron() and evaluate accuracy across k-folds

In [ ]:
# Test the Perceptron algorithm on the sonar dataset
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [76.81159420289855, 69.56521739130434, 72.46376811594203]
Mean Accuracy: 72.947%
